# Examining SN hubble residuals VS Host Galaxy Properties

In [ ]:
import os
from random import random

from astropy.table import Table
from matplotlib import pyplot as plt

import GCRCatalogs


In [ ]:
# Cosmo DC2 contains host galaxy properties
cosmo_gc = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_small')

# Truth Variable Summary catalog specifies variable type and ids of SN and hosts
truth_variable_summary_gc = GCRCatalogs.load_catalog('dc2_truth_run1.2_variable_summary')

# Truth Variable Light Curve catalog contains the "observed" light curves
truth_variable_gc = GCRCatalogs.load_catalog('dc2_truth_run1.2_variable_lightcurve')

# Truth Variable Light Curve catalog contains target meta data (ra, dec, redshift, etc.)
truth_static_gc = GCRCatalogs.load_catalog('dc2_truth_run1.2_static')


In [ ]:
def get_supernovae():
    """Return supernovae from the Truth Variable Summary Catalog
    
    Returns:
        A dictionary of arrays with keys 'galaxy_id', 'uniqueId', 'ra', and 'dec'
    """
    
    supernovae = truth_variable_summary_gc.get_quantities(
        ['galaxy_id', 'uniqueId', 'ra', 'dec'], 
        native_filters=['sn == 1', 'galaxy_id != -1']
    )
    
    return supernovae

In [ ]:
def write_snoopy_file(out_path, **kwargs):
    """Creates a snoopy input file out_path/name.txt
    
    Args:
        out_path         (str): Where to write the input file
        name             (str): The name of the target
        redshift       (float): The redshift of the target    
        ra             (float): The ra of the target
        dec            (float): The dec of the target
        <u, g, r, i, z> (dict): The date ('mjd') and magnitude ('mag')
    """
    
    file_text = f'{kwargs["name"]} {kwargs["redshift"]} {kwargs["ra"]} {kwargs["dec"]}\n'
    
    # Todo: Specify LSST filters including y
    band_names = {'u': 'u_s', 
                  'g': 'g_s',
                  'r': 'r_s',
                  'i': 'i_s',
                  'z': 'z_s'}
    
    for band, name in band_names.items():
        if band not in kwargs:
            continue
            
        file_text += f'filter {name}\n'
        
        for mjd, mag in zip(kwargs[band]['mjd'], kwargs[band]['mag']):
            # Todo: Specify the actual error
            file_text += f'{mjd} {mag} {0.01 * mag}\n'
    
    with open(out_path, 'w') as ofile:
        ofile.write(file_text)
    

In [ ]:
def create_snoopy_inputs(out_dir, supernovae):
    """Generate snoopy input files for supernova in the truth catalouge
    
    Args:
        out_dir     (str): The directory to write input files to
        supernovae (dict): Catalog data returned by get_supernovae
    """

    os.makedirs(out_dir, exist_ok=True)
    meta_table_path = os.path.join(out_dir, 'meta_data.csv')
    meta_table = Table(names=['target', 'stellat_mass', 'sfr'])
    
    iter_data = zip(supernovae['ra'], supernovae['dec'], supernovae['uniqueId'], supernovae['galaxy_id'])
    for ra, dec, sn_id, gal_id in iter_data:
        
        # Todo: Specify actual redshift
        redshift = 3 * random() #truth_static_gc.get_quantities(['redshift'], filters=[f'object_id == {gal_id}'])

        # Truth variable catalog is SQL based so individual queries go by quickly
        u = truth_variable_gc.get_quantities(['mjd', 'mag'], filters=['filter == 0'], native_filters=['sn == 1', f'uniqueId == {sn_id}'])
        g = truth_variable_gc.get_quantities(['mjd', 'mag'], filters=['filter == 1'], native_filters=['sn == 1', f'uniqueId == {sn_id}'])
        r = truth_variable_gc.get_quantities(['mjd', 'mag'], filters=['filter == 2'], native_filters=['sn == 1', f'uniqueId == {sn_id}'])
        i = truth_variable_gc.get_quantities(['mjd', 'mag'], filters=['filter == 3'], native_filters=['sn == 1', f'uniqueId == {sn_id}'])
        z = truth_variable_gc.get_quantities(['mjd', 'mag'], filters=['filter == 4'], native_filters=['sn == 1', f'uniqueId == {sn_id}'])
        y = truth_variable_gc.get_quantities(['mjd', 'mag'], filters=['filter == 5'], native_filters=['sn == 1', f'uniqueId == {sn_id}'])

        # Skip SN with no observations
        bands = (u, g, r, i , z, y)
        if sum((len(x['mjd']) for x in bands)) == 0:
            # print(f'No Observations: {sn_id}\n')  
            continue

        # meta_data = cosmo_gc.get_quantities(['baseDC2/obs_sfr', 'stellar_mass'], filters=[f'galaxy_id == {gal_id}'])
        # meta_table.add_row(... meta_data ...)
        
        out_path = os.path.join(out_dir, f'{sn_id}.txt')
        write_snoopy_file(
            out_path,
            u=u,
            g=g,
            r=r,
            i=i,
            z=z,
            name=sn_id,
            redshift=redshift,
            ra=ra,
            dec=dec
        )


In [ ]:
# Get supernova data from the truth variable summary catalog
supernovae = get_supernovae()
print(supernovae)

# Generate input files for snoopy
create_snoopy_inputs('./snoopy_data', supernovae)
